In [1]:
import os
import pandas as pd
from shutil import copyfile
import numpy as np
import math
import cv2
import decimal
from pyproj import Proj, transform
from PIL import Image
from sympy import symbols, Eq, solve

In [2]:
def find_lat_lon(img1, img2):

    a = df_book2.loc[df_book2['ns1:ImageId'] == img1]
    lat1 = a.iloc[0]['ns1:Latitude']
    lon1 = a.iloc[0]['ns1:Longitude']

    b = df_book2.loc[df_book2['ns1:ImageId'] == img2]
    lat2 = b.iloc[0]['ns1:Latitude']
    lon2 = b.iloc[0]['ns1:Longitude']

    
    #print('Latitude and Longitude ----', lat1, lon1, lat2, lon2)
    return lat1, lon1, lat2, lon2

def get_names(img_name1, img_name2):

    or_name1 = os.path.splitext(img_name1)[0]
    or_name2 = os.path.splitext(img_name2)[0]

    f_name1 = or_name1 + '.txt'
    f_name2 = or_name2 + '.txt'

    base_path = r'D:\Distance Measurement\GB71_experimental_data'

    text_file_path_l = os.path.join(base_path, f_name1)
    text_file_path_r = os.path.join(base_path, f_name2)

    img_name_l = os.path.join(base_path, img_name1)
    img_name_r = os.path.join(base_path, img_name2)
    
    return text_file_path_l, text_file_path_r, img_name_l, img_name_r

def calculate_distance_from_latlon(lat1, lon1, lat2, lon2):
    R = 6371.0
    lat2 = math.radians(lat2)
    lon2 = math.radians(lon2)
    lat1 = math.radians(lat1)
    lon1 = math.radians(lon1)
    
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = R * c * 1000

    #print('DISTANCE ----', distance)
    return distance 

def calculating_bbx_pixel(x_center, y_center, width, height, img_dir):
    x_min = x_center - width/2
    x_max = x_center + width/2
    
    y_min = y_center - height/2
    y_max = y_center + height/2
    
    im = Image.open(img_dir)
    img_width, img_height = im.size
    
    x_center_p = x_center * img_width
    y_center_p = y_center * img_height
    
    x_min_p = x_min * img_width
    x_max_p = x_max * img_width
    
    y_min_p = y_min * img_height
    y_max_p = y_max * img_height
    
    #print('PIXEL PARAMETERS ----', x_center_p, y_center_p, x_min_p, x_max_p, y_min_p, y_max_p)
    return x_center_p, y_center_p, x_min_p, x_max_p, y_min_p, y_max_p

def get_bbx_param(text_file_path):
    file = open(text_file_path)
    output_lst = file.read().split(' ')
    x_center, y_center, width, height = float(output_lst[1]), float(output_lst[2]), float(output_lst[3]), float(output_lst[4])
    
    #print('ANNOTED PARAMETERS ----', x_center, y_center, width, height)
    return x_center, y_center, width, height

def calculate_depth(baseline, focal_length, disparity):
    depth = baseline * focal_length / disparity
    #print('DEPTH ----', depth)
    return depth

In [3]:
df_req_signs = pd.read_excel('requested_signs10.xlsx')

In [4]:
book2_path = r'D:\Distance Measurement\More New Data\NL-EDEE-150611.DataPackage.csv'
df_book2 = pd.read_csv(book2_path)

In [5]:
all_fx = []
for index, row in df_req_signs.iterrows():
    img1 = row['Image_Name_1'][:8]
    img2 = row['Image_Name_2'][:8]
    lat1, lon1, lat2, lon2 = find_lat_lon(img1, img2)
    car1_car2 = calculate_distance_from_latlon(lat1, lon1, lat2, lon2)
    text_file_path_l, text_file_path_r, img_name_l, img_name_r = get_names(row['Image_Name_1'], row['Image_Name_2'])

    x_center_l, y_center_l, width_l, height_l = get_bbx_param(text_file_path_l)

    x_center_r, y_center_r, width_r, height_r = get_bbx_param(text_file_path_r)

    x_center_p_l, y_center_p_l, x_min_p_l, x_max_p_l, y_min_p_l, y_max_p_l = calculating_bbx_pixel(x_center_l, y_center_l, 
                                                                                                   width_l, height_l, img_name_l)

    x_center_p_r, y_center_p_r, x_min_p_r, x_max_p_r, y_min_p_r, y_max_p_r = calculating_bbx_pixel(x_center_r, y_center_r,
                                                                                                   width_r, height_r, img_name_r)
    
    #disparity = abs(x_center_p_l - x_center_p_r)
    disparity = ( (x_center_p_r - x_center_p_l)**2 + (y_center_p_r - y_center_p_l)**2 )**0.5
    baseline = car1_car2
    focal_length = 331

    depth = calculate_depth(baseline, focal_length, disparity)
    
    Actual_left_depth = row['Actual_left_depth']
    Actual_right_depth = row['Actual_right_depth']
    
    norm_error1 = abs(depth-Actual_left_depth)
    norm_error2 = abs(depth-Actual_right_depth)
    
    if (norm_error1>3) | (norm_error2>3) | (math.isnan(norm_error1)):
        continue
    
    z1 = Actual_left_depth
    z2 = Actual_right_depth
    d = disparity
    b = baseline
    
    fx = (z1*d)/b
    all_fx.append(fx)
    fx = (z2*d)/b
    all_fx.append(fx)

In [6]:
print(all_fx)

avg_fx = sum(all_fx)/len(all_fx)

avg_focal_length = avg_fx

print(avg_fx)

[298.59876601796145, 259.6968717777211, 313.06079034408583, 343.36950672055764, 268.1860197130236, 311.046783200841, 215.94653991697962, 316.2390673748396, 277.760507292105, 301.23752703178974, 316.8775213074695, 325.0312752399791]
295.58759799477946


In [8]:
#ox = ul - (fx*x)/z
all_ox = []
for index, row in df_req_signs.iterrows():
    img1 = row['Image_Name_1'][:8]
    img2 = row['Image_Name_2'][:8]
    lat1, lon1, lat2, lon2 = find_lat_lon(img1, img2)
    car1_car2 = calculate_distance_from_latlon(lat1, lon1, lat2, lon2)
    text_file_path_l, text_file_path_r, img_name_l, img_name_r = get_names(row['Image_Name_1'], row['Image_Name_2'])

    x_center_l, y_center_l, width_l, height_l = get_bbx_param(text_file_path_l)

    x_center_r, y_center_r, width_r, height_r = get_bbx_param(text_file_path_r)

    x_center_p_l, y_center_p_l, x_min_p_l, x_max_p_l, y_min_p_l, y_max_p_l = calculating_bbx_pixel(x_center_l, y_center_l, 
                                                                                                   width_l, height_l, img_name_l)

    x_center_p_r, y_center_p_r, x_min_p_r, x_max_p_r, y_min_p_r, y_max_p_r = calculating_bbx_pixel(x_center_r, y_center_r,
                                                                                                   width_r, height_r, img_name_r)
    
    #disparity = abs(x_center_p_l - x_center_p_r)
    disparity = ( (x_center_p_r - x_center_p_l)**2 + (y_center_p_r - y_center_p_l)**2 )**0.5
    baseline = car1_car2
    focal_length = avg_fx

    depth = calculate_depth(baseline, focal_length, disparity)
    
    Actual_left_depth = row['Actual_left_depth']
    Actual_right_depth = row['Actual_right_depth']
    
    norm_error1 = abs(depth-Actual_left_depth)
    norm_error2 = abs(depth-Actual_right_depth)
    
    if (norm_error1>3) | (norm_error2>3) | (math.isnan(norm_error1)):
        continue
    
    Actual_latitude = row['Actual_latitude']
    Actual_longitude = row['Actual_logitude']
    R = 6371 * 1000 
    #lat1 = math.radians(car1_lat)
    #lon1 = math.radians(car1_lon)
    lat1 = math.radians(lat1)
    lon1 = math.radians(lon1)
    lat2 = math.radians(lat2)
    lon2 = math.radians(lon2)

    x11 = R * math.cos(lat1) * math.cos(lon1)
    y11 = R * math.cos(lat1) * math.sin(lon1)
    z11 = R * math.sin(lat1)
    
    print(x11)
    
    x22 = R * math.cos(lat2) * math.cos(lon2)
    y22 = R * math.cos(lat2) * math.sin(lon2)
    z22 = R * math.sin(lat2)

    #x1o, y1o, z1o = x11+x1, y11+y1, z11+z
    #x1o, y1o, z1o = x11+x1, y11+y1, z11+z

    Actual_latitude = math.radians(Actual_latitude)
    Actual_longitude = math.radians(Actual_longitude)
    xo = R * math.cos(Actual_latitude) * math.cos(Actual_longitude)
    yo = R * math.cos(Actual_latitude) * math.sin(Actual_longitude)
    zo = R * math.sin(Actual_latitude)
    
    x1 = (xo - x11)
    y1 = (yo - y11)
    z1 = (zo - z11)
    
    x2 = (xo - x22)
    y2 = (yo - y22)
    z2 = (zo - z22)
    
    ul = x_center_p_l
    ur = x_center_p_r
    b = car1_car2
    d = disparity
    
    ox = ul - (d*x1/b)
    all_ox.append(ox)
    ox = ur - (d*x2/b)
    all_ox.append(ox)

3898900.506254729
3898912.09772243
3898793.619421212
3895872.5178596997
3898793.619421212
3898873.594907834


In [9]:
print(all_ox)

avg_ox = sum(all_ox)/len(all_ox)

print(avg_ox)

[520.6162493206172, 124.6628307072982, 299.31127640111214, 240.3320867927241, 397.5683117265734, 366.54418396572834, 250.0944036816084, 590.5156793512288, 192.81670467322925, -56.60049994924097, 312.8625765335043, 227.4713162105866]
288.8495932845808


In [10]:
errors1 = []
errors2 = []
norm_errors1 = []
norm_errors2 = []
yaw_errors1 = []
yaw_errors2 = []

for index, row in df_req_signs.iterrows():
    img1 = row['Image_Name_1'][:8]
    img2 = row['Image_Name_2'][:8]
    lat1, lon1, lat2, lon2 = find_lat_lon(img1, img2)
    car1_car2 = calculate_distance_from_latlon(lat1, lon1, lat2, lon2)
    text_file_path_l, text_file_path_r, img_name_l, img_name_r = get_names(row['Image_Name_1'], row['Image_Name_2'])

    x_center_l, y_center_l, width_l, height_l = get_bbx_param(text_file_path_l)

    x_center_r, y_center_r, width_r, height_r = get_bbx_param(text_file_path_r)

    x_center_p_l, y_center_p_l, x_min_p_l, x_max_p_l, y_min_p_l, y_max_p_l = calculating_bbx_pixel(x_center_l, y_center_l, 
                                                                                                   width_l, height_l, img_name_l)

    x_center_p_r, y_center_p_r, x_min_p_r, x_max_p_r, y_min_p_r, y_max_p_r = calculating_bbx_pixel(x_center_r, y_center_r,
                                                                                                   width_r, height_r, img_name_r)
    
    #disparity = abs(x_center_p_l - x_center_p_r)
    disparity = ( (x_center_p_r - x_center_p_l)**2 + (y_center_p_r - y_center_p_l)**2 )**0.5
    baseline = car1_car2
    focal_length = avg_focal_length #331

    depth = calculate_depth(baseline, focal_length, disparity)
    
    Actual_left_depth = row['Actual_left_depth']
    Actual_right_depth = row['Actual_right_depth']
    
    norm_error1 = abs(depth-Actual_left_depth)
    norm_error2 = abs(depth-Actual_right_depth)
    
    #if (norm_error1>3) | (norm_error2>3) | (math.isnan(norm_error1)):
        #continue
        
    norm_errors1.append(norm_error1)
    norm_errors2.append(norm_error2)

    dis = ( (x_center_p_l)**2 + (y_center_p_l)**2 )**0.5
    predicted_left_depth = (depth*dis)/focal_length
    
    dis = ( (x_center_p_r)**2 + (y_center_p_r)**2 )**0.5
    predicted_right_depth = (depth*dis)/focal_length
    
    error1 = abs(predicted_left_depth-Actual_left_depth)
    error2 = abs(predicted_right_depth-Actual_right_depth)
    
    errors1.append(error1)
    errors2.append(error2)

    a = df_book2.loc[df_book2['ns1:ImageId'] == img1]
    yaw1 = a.iloc[0]['ns1:RecorderDirection']
    a = df_book2.loc[df_book2['ns1:ImageId'] == img2]
    yaw2 = a.iloc[0]['ns1:RecorderDirection']
    
    ur = x_center_p_r
    vr = y_center_p_r
    b = car1_car2
    d = disparity
    ox = avg_ox
    z = depth
    del_yaw = math.radians(yaw1 - yaw2)
    #del_yaw = yaw1- yaw2

    x2 = b*(ur-ox)/d
    #x2

    corrected_depth = - ( (1+x2) * z * z * del_yaw ) / b 
    depth = depth + corrected_depth
    
    yaw_error1 = abs(depth-Actual_left_depth)
    yaw_error2 = abs(depth-Actual_right_depth)
    
    yaw_errors1.append(yaw_error1)
    yaw_errors2.append(yaw_error2)

In [11]:
print(norm_errors1)
print(norm_errors2)

[0.06727213369749308, 0.3394145781186815, 1.0009902498436745, nan, nan, nan, 8.238787178131243, 11.557964853650315, 5.442449883448514, 1.7450168221761837, 52.982248967218474, 3.4865538915755985, 0.6287525639052962, 0.2867316931250361]
[0.8018302910983568, 0.928157606672781, 0.564730013091916, nan, nan, nan, 3.695575442364433, 7.360256315762655, 9.781103516556895, 0.45249476012306644, 57.85351987660707, 1.4869444019399625, 0.19927016848409274, 0.3965460703802064]
